# **Importing libraries**

In [ ]:
pip install bs4

In [ ]:
# for web scraping
import requests # required for HTTP request
import bs4
from bs4 import BeautifulSoup # required for HTML and XML parsing                                                            
import pandas as pd 
import time
import re
import math 
import pickle

# for text processing
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn



# **Downloading and parsing Amazon pages**

In [ ]:
#Create lists to store needed info
descriptions=[] # List to store product descriptions
prices=[] # List to store price of the product
prime=[] #Binary list (prime=1, not prime=0)
urls=[] #List to store url of the product page
ratings=[] # List to store ratings of the product

n_pages = 7 # no of pages to scrape in the website

for page in range(1, n_pages+1):
  headers = {"User-Agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 \
  (KHTML, like Gecko) Chrome/41.0.2228.0",} 
  r = requests.get("https://www.amazon.it/s?k=computer&page="+str(page), headers=headers)#,proxies=proxies)
  content = r.content
  soup = BeautifulSoup(content)

  #Parse all items
  for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 \
  sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col \
  sg-col-4-of-20 sg-col-4-of-32'}): 
    #print(d)
    #this class is the div containing the product name, price, rating...
    description = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
    price = d.find('span', attrs={'class':'a-offscreen'})
    is_prime = d.find('span', attrs={'class':'aok-relative s-icon-text-medium s-prime'})
    url = d.find('a', attrs={'class':'a-link-normal a-text-normal'})
    rating = d.find('span', attrs={'class':'a-icon-alt'})
    
    #Add items information
    if description is not None:
      descriptions.append(description.text)
      print(description.text)
    else:
      descriptions.append("unknown-description")

    if price is not None:
      prices.append(price.text)
      print(price.text)
    else:
      prices.append('0 €')

    if is_prime is not None:
      prime.append('1')
      print('1')
    else:
      prime.append('0')
      print('0')

    if url is not None:
      urls.append(url['href'])
      print(url['href'])
    else:
      urls.append("unknown-url")

    if rating is not None:
      score = re.findall("[+-]?\d+\,\d+", rating.text)
      #score = score[0].replace(',','.')
      ratings.append(score[0])
      print(score[0])
    else:
      ratings.append('-1')

   
  time.sleep(100) #stop the code for 100 seconds after each page
  print("\n\n PAGE ", page, " DOWNLOADED! \n\n\n")


#Add products to tsv file
df = pd.DataFrame({'Description':descriptions, 'Price':prices, 'Prime':prime, 'Url':urls, 'Ratings':ratings})
df.to_csv('computer.tsv', index=False, encoding='utf-8', sep = '\t')

Computer portatile da 15,6 pollici, CPU Intel N3350, Windows 10 Pro OS,4 GB RAM 64 GB SSD, Full HD 1920 x 1080
298,00 €
1
/Computer-portatile-pollici-Intel-Windows/dp/B08KFZNGGH/ref=sr_1_1?dchild=1&keywords=computer&qid=1604664192&sr=8-1
PC DELL 7010 SFF Intel Core i5 3470 3.20Ghz/RAM 8GB/500GB/DVD+RW/WIN 10 PRO (Ricondizionato)
167,99 €
0
/DELL-Intel-3-20Ghz-500GB-Ricondizionato/dp/B07RK9N3WM/ref=sr_1_2?dchild=1&keywords=computer&qid=1604664192&sr=8-2
4,3
Laptop da 14 pollici (Intel Celeron a 64 bit, 4 GB di RAM DDR3, 64 GB di eMMC, batteria da 10000 Mah, webcam HD, sistema operativo Windows 10 preinstallato, display IPS 1366 * 768 FHD) Notebook
254,99 €
1
/pollici-batteria-operativo-preinstallato-Notebook/dp/B08BHRW3KB/ref=sr_1_3?dchild=1&keywords=computer&qid=1604664192&sr=8-3
2,2
HUAWEI MateBook 14 Laptop, Ultrabook DisplayFullView 2K , AMD Ryzen 7 4800H, 16GB RAM, SSD da 512GB PCIe , Fingerabdrucksensor, Huawei Share, Windows 10 Home, Space Gray, KelvinL-WFE9B
1.099,00 €
1
/HUAWEI

# **Preprocessing product descriptions**




Let's read and look at our product database:

In [ ]:
df = pd.read_csv(r'computer.tsv', sep='\t' )
pd.set_option('display.max_colwidth', -1)

print(df.shape)
df.tail(10)

(306, 5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Description,Price,Prime,Url,Ratings
296,"Schenker Via 14 Grigio Computer Portatile 35,6 cm (14"") 1920 x 1080 Pixel Intel® Core i7 di Decima Generazione 16 GB DDR4-SDRAM 512 GB SSD Wi-Fi 6 (802.11ax) Windows 10 PRO Via 14,","1.575,56 €",1,/Schenker-Computer-Portatile-Generazione-DDR4-SDRAM/dp/B084ZZ6BYT/ref=sr_1_298?dchild=1&keywords=computer&qid=1604664803&sr=8-298,-1
297,"Murder, She Wrote Season 8",0 €,0,/gp/video/ssoredirect/?ru=https%3A%2F%2Fapp.primevideo.com%2Fdetail%3Fgti%3Damzn1.dv.gti.06b3e90e-2cea-056f-bc7a-57e5f7322f7c%26ref_%3Ddvm_src_ret_it_xx_s&page-type-id=B083D9NCVT&ref=sr_1_299&keywords=computer&qid=1604664803&sr=8-299,"4,8"
298,"TECLAST Tablet PC 2 in 1 Laptop X6pro 12.6 Pollici 2880x1928 IPS Windows 10 Intel Core M3-7Y30, 8GB ROM+256GB SSD, 2+5 MP Telecamera+Tipe-C+Bluetooth+GPS+WiFi+38000 mWh (Non incluse Stilo e Tastiera)","459,49 €",1,/TECLAST-2880x1928-Telecamera-Bluetooth-Tastiera/dp/B089K65YZ3/ref=sr_1_300?dchild=1&keywords=computer&qid=1604664803&sr=8-300,"2,0"
299,"Replitel Webcam with Microphone, 1080P HD Streaming USB Computer Webcam for PC Video Conferencing/Calling/Gaming, Laptop/Desktop Mac, Skype/Youtube/Zoom/Facetime-Plug And Play (Black)","18,99 €",0,/Replitel-Microphone-Streaming-Conferencing-Facetime-Plug/dp/B08L5W7MHJ/ref=sr_1_301?dchild=1&keywords=computer&qid=1604664803&sr=8-301,"4,7"
300,"HP - PC Pavilion 15-eh0000sl Notebook, AMD Ryzen 5 4500U, RAM 8 GB, SSD 512 GB, Grafica AMD Graphics, Windows 10 Home, Schermo 15.6"" FHD IPS, Lettore Impronte digitali, Webcam, Argento","699,99 €",1,/HP-Pavilion-15-eh0000sl-Notebook-Graphics/dp/B08HQRJ1JV/ref=sr_1_302?dchild=1&keywords=computer&qid=1604664803&sr=8-302,-1
301,"Lenovo IdeaPad 5 Notebook, Display 14"" Full HD AntiGlare, Processore AMD Ryzen 5 4500U, 512 GB SSD, RAM 8 GB, Fingerprint, Windows 10, Light Teal",0 €,0,/Lenovo-Notebook-AntiGlare-Processore-Fingerprint/dp/B089NVGRGC/ref=sr_1_303?dchild=1&keywords=computer&qid=1604664803&sr=8-303,"4,6"
302,"Aigoss Webcam 1080p Full HD con Microfono, PC Laptop Desktop Computer USB per Videochiamate, Studio, Conferenza, Registrazione, Gioca a Giochi e Lavoro a Casa","35,99 €",1,/Aigoss-Microfono-Videochiamate-Conferenza-Registrazione/dp/B08BNM52L8/ref=sr_1_304?dchild=1&keywords=computer&qid=1604664803&sr=8-304,"4,8"
303,"Looffy Pendrive 3.0 per iphone ipad 128gb, Chiavetta USB 4 in 1 per OTG Telefono Android Tipo C USB C 128gb, Memoria USB per Computer OTG Smartphone Laptop","39,79 €",1,/Looffy-Pendrive-Chiavetta-Telefono-Smartphone/dp/B08FBTQS1L/ref=sr_1_305?dchild=1&keywords=computer&qid=1604664803&sr=8-305,"4,5"
304,"Lenovo IdeaCentre AIO A340-24IWL Computer desktop All-in-One con Processore Intel Core i5-8265U, 8 GB di RAM, 128 GB SSD, 1 TB HDD, masterizzatore DVD, Intel UHD, Windows 10 Home 64, nero","812,31 €",1,/Lenovo-Ideacentre-AIO-A340-24IWL/dp/B07VTNXXNC/ref=sr_1_310?dchild=1&keywords=computer&qid=1604664803&sr=8-310,-1
305,NEWHEY Custodia per Macbook Air 13 Pollici Impermeabile Custodia PC 13-14 Pollici Antiurto Porta Computer Custodia Borsa con Piccolo Caso Rosa,"14,86 €",1,/NEWHEY-Custodia-Impermeabile-Antiurto-Computer/dp/B08DJ6FHJY/ref=sr_1_311?dchild=1&keywords=computer&qid=1604664803&sr=8-311,"4,4"


In [ ]:
nltk.download('wordnet')
nltk.download('omw')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
set(stopwords.words('italian'))

Let's define a function that preprocesses our product description, using nltk library:

In [ ]:
#this function preprocesses a text (either query or product description document)
def preprocess(text):

  tokens = nltk.word_tokenize(text, language='italian')  #tokenize
  tokens=[token.lower() for token in tokens if token.isalpha() or token.isdigit()] #remove punctuations

  italian_stopwords = stopwords.words('italian')
  tokens = [w for w in tokens if w not in italian_stopwords]  #remove stopwords)

  ita_stemmer = nltk.stem.snowball.ItalianStemmer()
  tokens = [ita_stemmer.stem(w) for w in tokens] #stem
  
  return tokens

In [ ]:
corpus = df['Description'].dropna().apply(lambda x: preprocess(x))
corpus.tail()

301    [lenov, ideapad, 5, notebook, display, 14, full, hd, antigl, processor, amd, ryzen, 5, 512, gb, ssd, ram, 8, gb, fingerprint, windows, 10, light, teal]       
302    [aigoss, webcam, full, hd, microf, pc, laptop, desktop, computer, usb, videochiam, stud, confert, registr, gioc, gioc, lavor, cas]                            
303    [looffy, pendriv, iphon, ipad, chiavett, usb, 4, 1, otg, telef, android, tip, usb, memor, usb, computer, otg, smartphon, laptop]                              
304    [lenov, ideacentr, aio, computer, desktop, processor, intel, cor, 8, gb, ram, 128, gb, ssd, 1, tb, hdd, masterizz, dvd, intel, uhd, windows, 10, hom, 64, ner]
305    [newhey, custod, macbook, air, 13, pollic, imperm, custod, pc, pollic, antiurt, port, computer, custod, bors, piccol, cas, ros]                               
Name: Description, dtype: object

# **Building the inverted index**

After saving the documents in an inverted index, we use vector space model as retrieval model of our search engine.

We need an index such that: term --> (docID, tf-idf(doc, term))


because our algorithm for CosineScore(q) is such that for each QUERY TERM t, we:
- compute tf-idf weight(t,q)
- for each entry of the posting list of term t
-             scores[d]= weight(t,d) weight(t,q)


then normalize score of each document by document lenght

**Constructing the inverted index basing on Term Frequencies**

In [ ]:
dictionary = dict()

#Defines Term frequencies in an inverted index structure
def II_Tf(docID,description, dictionary):
  for term in description:
    if term in dictionary:
      if docID not in dictionary[term]:
        dictionary[term][docID] = 1
      else:
        dictionary[term][docID] += 1
    else:
      #Add the term to the vocabulary
      dictionary[term] = {}
      dictionary[term][docID] = 1
  return dictionary

id = 0
for row in corpus:
  #print(row)
  dictionary = II_Tf(id, row, dictionary)
  id += 1
print(dictionary)

{'computer': {0: 1, 5: 1, 8: 2, 12: 1, 15: 1, 16: 2, 19: 1, 22: 1, 24: 1, 26: 1, 27: 1, 28: 1, 29: 1, 36: 1, 38: 1, 40: 1, 45: 1, 47: 1, 48: 1, 51: 1, 52: 1, 53: 1, 54: 1, 57: 1, 58: 1, 61: 1, 62: 1, 68: 1, 70: 1, 74: 1, 75: 1, 77: 1, 78: 1, 83: 1, 86: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 95: 1, 97: 1, 99: 1, 100: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 112: 1, 115: 1, 117: 1, 118: 1, 119: 1, 120: 1, 122: 1, 123: 1, 124: 1, 125: 1, 128: 1, 129: 1, 130: 1, 131: 1, 132: 1, 137: 1, 138: 1, 140: 1, 141: 1, 142: 1, 145: 1, 146: 1, 147: 1, 149: 1, 152: 2, 153: 1, 156: 1, 157: 1, 159: 1, 162: 1, 165: 1, 166: 1, 172: 1, 173: 1, 177: 1, 181: 1, 182: 1, 187: 1, 188: 1, 189: 1, 190: 1, 192: 2, 194: 1, 196: 1, 198: 1, 201: 1, 202: 1, 203: 1, 204: 1, 205: 1, 206: 1, 207: 1, 211: 1, 212: 1, 213: 1, 216: 1, 218: 1, 220: 1, 222: 1, 223: 3, 230: 1, 234: 1, 235: 1, 237: 1, 238: 1, 240: 1, 243: 1, 244: 1, 246: 1, 247: 1, 248: 1, 249: 1, 250: 1, 251: 1, 252: 1, 253: 1, 254: 1, 255: 1, 258: 1, 

**Transforming the Tf Inverted Index into a Tf-Idf Inverted Index**

In [ ]:
#If you want to see the terms in the dictionary, here is the list
#for term in dictionary:
#  print(term)

#If you want to print the postings lists of each term, here is the list
#for term in dictionary:
#  print(term, dictionary[term])

N = corpus.count() #306 docs in total
for term in dictionary:
  df = (len(dictionary[term])) #number of docs containing that term
  #Update Tf as Tf-Idf
  for doc in dictionary[term]:
    #dictionary[term][doc] is the tf
    idf = math.log(N/df)
    dictionary[term][doc] = dictionary[term][doc] * idf

print(dictionary)


{'computer': {0: 0.7196387960069216, 5: 0.7196387960069216, 8: 1.4392775920138432, 12: 0.7196387960069216, 15: 0.7196387960069216, 16: 1.4392775920138432, 19: 0.7196387960069216, 22: 0.7196387960069216, 24: 0.7196387960069216, 26: 0.7196387960069216, 27: 0.7196387960069216, 28: 0.7196387960069216, 29: 0.7196387960069216, 36: 0.7196387960069216, 38: 0.7196387960069216, 40: 0.7196387960069216, 45: 0.7196387960069216, 47: 0.7196387960069216, 48: 0.7196387960069216, 51: 0.7196387960069216, 52: 0.7196387960069216, 53: 0.7196387960069216, 54: 0.7196387960069216, 57: 0.7196387960069216, 58: 0.7196387960069216, 61: 0.7196387960069216, 62: 0.7196387960069216, 68: 0.7196387960069216, 70: 0.7196387960069216, 74: 0.7196387960069216, 75: 0.7196387960069216, 77: 0.7196387960069216, 78: 0.7196387960069216, 83: 0.7196387960069216, 86: 0.7196387960069216, 88: 0.7196387960069216, 89: 0.7196387960069216, 90: 0.7196387960069216, 91: 0.7196387960069216, 92: 0.7196387960069216, 95: 0.7196387960069216, 97: 0

Save the dictionary in a file (non human readable)

In [ ]:
ii_file = open("inverted_index.pkl", "wb")
pickle.dump(dictionary, ii_file)
ii_file.close()

# **Query Processing and Answering**

First, we read the inverted index stored in the file.

In [ ]:
ii_file = open("inverted_index.pkl", "rb")
inverted_index = pickle.load(ii_file)
print(inverted_index)

{'computer': {0: 0.7196387960069216, 5: 0.7196387960069216, 8: 1.4392775920138432, 12: 0.7196387960069216, 15: 0.7196387960069216, 16: 1.4392775920138432, 19: 0.7196387960069216, 22: 0.7196387960069216, 24: 0.7196387960069216, 26: 0.7196387960069216, 27: 0.7196387960069216, 28: 0.7196387960069216, 29: 0.7196387960069216, 36: 0.7196387960069216, 38: 0.7196387960069216, 40: 0.7196387960069216, 45: 0.7196387960069216, 47: 0.7196387960069216, 48: 0.7196387960069216, 51: 0.7196387960069216, 52: 0.7196387960069216, 53: 0.7196387960069216, 54: 0.7196387960069216, 57: 0.7196387960069216, 58: 0.7196387960069216, 61: 0.7196387960069216, 62: 0.7196387960069216, 68: 0.7196387960069216, 70: 0.7196387960069216, 74: 0.7196387960069216, 75: 0.7196387960069216, 77: 0.7196387960069216, 78: 0.7196387960069216, 83: 0.7196387960069216, 86: 0.7196387960069216, 88: 0.7196387960069216, 89: 0.7196387960069216, 90: 0.7196387960069216, 91: 0.7196387960069216, 92: 0.7196387960069216, 95: 0.7196387960069216, 97: 0

Using this algorithm, we compute cosine scores only for the docs containg at least one term appearing in the query, and not for every document in the corpus. We use the inverted index to find docID containing such query terms (inspecting postings lists of query terms).

In [ ]:
#This function computes the cosine similarity score between the query and the documents
#containing at least one query term
#It returns the list of docIDs and their scores (but only if score is not 0)
#q is an already preprocessed query

def CosineScore(query):

  #Initialization
  results = []
  scores = [] #empty list of results to be returned (docIDs)
  n_docs = len(df)
  Scores = [0] * n_docs
  Length = [0] * n_docs
  
  for i in range(0,n_docs):
    Length[i]=len(df['Description'][i].split())
  
  #Binary Vectorization for the query => we eliminate duplicate terms
  query = list(dict.fromkeys(query))

  for term in query:
    #fetch correspondent postings list
    for docID in dictionary[term]:
      wtd = dictionary[term][docID]
      Scores[docID] += wtd # *1 because we are using a Binary Vectorization for the query
  
  for d in range(0,n_docs):
    if(Scores[d] != 0):
      Scores[d] = Scores[d] / Length[d] #Normalize scores by lenght
      results.append(d)
      scores.append(Scores[d])
   
  return results, scores



# This function prints the top 10 related products (ordered by descending cosine score)
# the first retrieved product is the MOST RELATED PRODUCT TO THE QUERY
def Answer(q):
  q=preprocess(q) 
  
  results, scores = CosineScore(q)
  res = pd.DataFrame({'DocID':results, 'Scores':scores})

  #sort scores in descending order
  res = res.sort_values(by ='Scores', ascending=False)
  #retrieve just the first 10 items (if you want just THE MOST RELATED, change this line to:
  #res = res[:1] )
  res = res[:10]

  #print all information related to each product
  print("\n\nAVAILABLE PRODUCTS:")
  for id in res['DocID']:
    print(df.iloc[id].to_string(),"\n\n")




Let's test the search engine with your query!

In [ ]:
q = input("Search for a product . . . ")
Answer(q)

Search for a product . . . NEWHEY Custodia per Macbook Air 13 Pollici Impermeabile Custodia PC 13-14 Pollici Antiurto Porta Computer Custodia Borsa con Piccolo Caso Rosa	


AVAILABLE PRODUCTS:
Description    NEWHEY Custodia per Macbook Air 13 Pollici Impermeabile Custodia PC 13-14 Pollici Antiurto Porta Computer Custodia Borsa con Piccolo Caso Rosa
Price          14,86 €                                                                                                                                       
Prime          1                                                                                                                                             
Url            /NEWHEY-Custodia-Impermeabile-Antiurto-Computer/dp/B08DJ6FHJY/ref=sr_1_311?dchild=1&keywords=computer&qid=1604664803&sr=8-311                 
Ratings        4,4                                                                                                                                            


Description   

**Further improvements**: 

- consider also synonyms, for example 'laptop' and 'portatile'

- consider ngrams (ex: '8 gb' instead of '8' and 'gb)

- correct misspellings in the query ...